In [2]:
using Pkg
Pkg.activate("../ast_obs")

 Activating environment at `C:\Users\shubh\Documents\AutonomousRiskFramework\ast_obs\Project.toml`


In [2]:
using Markdown
using InteractiveUtils
using Interact
using LinearAlgebra
using Plots
using Statistics
using Distributions, Parameters, Random, Latexify
using AutomotiveSimulator, AutomotiveVisualization
using POMDPs, POMDPPolicies, POMDPSimulators

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5859882815285967630\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-11443234297954375440\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5859882815285967630\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
using POMDPStressTesting

In [4]:
using AdversarialDriving

In [31]:
using Debugger

┌ Info: Precompiling Debugger [31a5f54b-26ea-5ae9-a837-f05ce5417438]
└ @ Base loading.jl:1278


In [5]:
# using Revise

In [6]:
# using ObservationModels

In [7]:
Base.rand(rng::AbstractRNG, s::Scene) = s

In [8]:
# Pkg.develop(url="https://github.com/sisl/AdversarialDriving.jl")
# Pkg.develop(PackageSpec(url="/home/users/shubhgup/Codes/julia_learn/scripts/AutonomousRiskFramework/AdversarialDriving.jl"))
# Pkg.develop(PackageSpec(url="/home/users/shubhgup/Codes/julia_learn/scripts/AutonomousRiskFramework/RiskSimulator.jl"))
# Pkg.develop(PackageSpec(url="/home/users/shubhgup/Codes/julia_learn/scripts/AutonomousRiskFramework/STLCG.jl"))

In [9]:
using CrossEntropyMethod

### Function Definitions

In [10]:
# # The function that propagates the PedestrianControl action
# function AdversarialDriving.propagate(ped::Entity{NoisyPedState, D, I}, action::PedestrianControl, roadway::Roadway, Δt::Float64) where {D, I}
#     starting_lane = laneid(ped)
#     vs_entity = Entity(ped.state.veh_state, ped.def, ped.id)
#     a_lat_lon = reverse(action.a + action.da)
#     vs = propagate(vs_entity, LatLonAccel(a_lat_lon...), roadway, Δt)
#     upd_noise = Noise((ped.state.noise.pos[1] + action.noise.pos[1] + action.noise.vel*Δt, ped.state.noise.pos[2] + action.noise.pos[2]), action.noise.vel)
#     nps = NoisyPedState(AdversarialDriving.set_lane(vs, laneid(ped), roadway), upd_noise)
#     @assert starting_lane == laneid(nps)
#     nps
# end

In [11]:
# # The function to get GPS measurements for pedestrian, compute state estimate and update noise
# function measure_gps(ped::Entity{NoisyPedState, D, I}, fixed_sat::Array{Satellite}, noise::Array{Float64}) where {D, I}
#     ped_pos = posg(ped)
    
#     ranges = Union{Missing, GPSRangeMeasurement}[]
    
#     for i in 1:length(fixed_sat)
#         satpos = fixed_sat[i].pos
#         if fixed_sat[i].visible==true
#             range = hypot(ped_pos.x - satpos.x, ped_pos.y - satpos.y, satpos.z)
#             push!(ranges, GPSRangeMeasurement(range=range, noise=noise[i]))
#         else
#             push!(ranges, missing)
#         end
#     end
    
#     ranges
# end

In [12]:
# measure_gps(ped_state, fixed_sat, [0.0, 0.0, 0.0, 0.0])

In [13]:
# function update_all_measurements(ped::Entity{NoisyPedState, D, I}, all_measurements) where {D, I}
#     single_time_measurements = Array{Measurement}[] 
# end

In [14]:
# Instructions for rendering the noisy pedestrian
function AutomotiveVisualization.add_renderable!(rendermodel::RenderModel, ped::Entity{NoisyPedState, VehicleDef, Int64})
    reg_ped = Entity(ped.state.veh_state, ped.def, ped.id)
    add_renderable!(rendermodel, FancyPedestrian(ped=reg_ped))
    noisy_ped = Entity(noisy_entity(ped, ad_mdp.roadway).state.veh_state, ped.def, ped.id)
    ghost_color = weighted_color_mean(0.3, colorant"blue", colorant"white")
    add_renderable!(rendermodel, FancyPedestrian(ped=noisy_ped, color=ghost_color))
    return rendermodel
end

In [15]:
# function AdversarialDriving.update_adversary!(adversary::Agent, action::Disturbance, s::Scene)
#     index = findfirst(id(adversary), s)
#     isnothing(index) && return nothing # If the adversary is not in the scene then don't update
#     adversary.model.next_action = action # Set the adversaries next action
#     # veh = s[index] # Get the actual entity
#     # state_type = typeof(veh.state) # Find out the type of its state
#     # s[index] =  Entity(state_type(veh.state, noise = action.noise), veh.def, veh.id) # replace the entity in the scene
# end

In [16]:
# # Function to assign action as state noise directly
# function AdversarialDriving.localize(ped::Entity{NoisyPedState, D, I}, action::PedestrianControl, meas, roadway::Roadway) where {D,I}
#     upd_noise = Noise(pos = (action.noise.pos[1], action.noise.pos[2]), vel = action.noise.vel)
#     nps = NoisyPedState(ped.state.veh_state, upd_noise)
#     nps
# end

In [17]:
# # Function to track the state of a pedestrian using velocity measurements
# function AdversarialDriving.localize(ped::Entity{NoisyPedState, D, I}, action::PedestrianControl, meas, roadway::Roadway) where {D,I}
#     # TODO: Replace with a higher-fidelity localization routine
#     ped_noise = noise(ped)
#     upd_noise = Noise(pos = (ped_noise.pos[1] + action.noise.vel*0.1, ped_noise.pos[2]), vel = action.noise.vel)
#     nps = NoisyPedState(ped.state.veh_state, upd_noise)
#     nps
# end

In [18]:
# # # Function to estimate the state of a pedestrian using GPS measurements
# function localize(ped::Entity{NoisyPedState, D, I}, action::PedestrianControl, meas, roadway::Roadway) where {D,I}
#     # TODO: Replace with a higher-fidelity localization routine
#     last_meas = last(meas)
#     gps_fix = GPS_fix(last_meas)
#     ped_pos = posg(ped)
#     upd_noise = Noise(pos = (gps_fix[1] - ped_pos.x, gps_fix[2] - ped_pos.y), vel = action.noise.vel)
#     nps = NoisyPedState(ped.state.veh_state, upd_noise)
#     nps
# end

In [19]:
# function logprob(ped::Entity{NoisyPedState, D, I}, action::PedestrianControl, meas, roadway::Roadway) where {D,I}
    
# end

### Main

In [20]:
# init_noise = Noise(pos = (0, 0), vel = 0, gps_range = [0.0, 0.0, 0.0, 0.0, 0.0])
# cont_noise = Noise(pos = (-7, 0), vel = 0, gps_range = [0.0, 0.0, 0.0, 0.0, 0.0])

In [21]:
init_noise = Noise(pos = (0, 0), vel = 0)
cont_noise = Noise(pos = (-7, 0), vel = 0)

Noise
  pos: VecE2{Float64}
  vel: Float64 0.0


In [22]:
sut_agent = BlinkerVehicleAgent(get_ped_vehicle(id=1, s=5.0, v=15.0),
    TIDM(ped_TIDM_template, noisy_observations=true));

In [23]:
get_pedestrian_noisy(;id::Int64, s::Float64, v::Float64, noise::Noise) = (rng::AbstractRNG = Random.GLOBAL_RNG) -> NoisyPedestrian(roadway = AdversarialDriving.ped_roadway, lane = 2, s=s, v=v, id=id, noise=noise)

get_pedestrian_noisy (generic function with 1 method)

In [24]:
adv_ped = NoisyPedestrianAgent(get_pedestrian_noisy(id=2, s=7.0, v=2.0, noise=init_noise), AdversarialPedestrian());

In [25]:
ad_mdp = AdversarialDrivingMDP(sut_agent, [adv_ped], ped_roadway, 0.1);

In [38]:
ped_state, veh_state = initialstate(ad_mdp)

LoadError: MethodError: no method matching iterate(::POMDPModelTools.Deterministic{Scene{Entity{S,VehicleDef,Int64} where S}})
Closest candidates are:
  iterate(!Matched::Tables.DictRowTable) at /scratch/users/shubhgup/julia/.julia/packages/Tables/iG2a3/src/dicts.jl:120
  iterate(!Matched::Tables.DictRowTable, !Matched::Any) at /scratch/users/shubhgup/julia/.julia/packages/Tables/iG2a3/src/dicts.jl:120
  iterate(!Matched::Pandas.HDFStore, !Matched::Any...) at /scratch/users/shubhgup/julia/.julia/packages/Pandas/rAPmB/src/Pandas.jl:63
  ...

In [ ]:
noisy_action = Disturbance[PedestrianControl(a=VecE2(0, 0), da=VecE2(0, 0), noise=cont_noise)]

In [ ]:
# Behavior with noise
hist_noise = POMDPSimulators.simulate(HistoryRecorder(), ad_mdp,
    FunctionPolicy((s) -> noisy_action));

In [ ]:
map(x -> (x.entities[1].state.veh_state.posG[2], AdversarialDriving.noisy_entity(x.entities[1], ad_mdp.roadway).state.veh_state.posG[2], noise(x.entities[1]).pos[1]) , POMDPSimulators.stepthrough(ad_mdp, FunctionPolicy((s) -> noisy_action), "s", max_steps=20))

In [ ]:
ad_scenes_noise = state_hist(hist_noise);

In [ ]:
# t = 10
@manipulate for t=1:length(ad_scenes_noise)
    AutomotiveVisualization.render([ad_mdp.roadway, crosswalk, ad_scenes_noise[t]])
end

#### Importance distribution

In [ ]:
struct INormal_Uniform{T<:Real} <: ContinuousUnivariateDistribution
    mu::Float64
    sigma::Float64
    mu_sup::Float64
    sup::Float64
    INormal_Uniform(mu::T, sigma::T, mu_sup::T, sup::T) where {T<:Real} = new{T}(Float64(mu), Float64(sigma), Float64(mu_sup), Float64(sup))
end

In [ ]:
INormal_Uniform(mu, sigma, mu_sup) = INormal_Uniform(Float64(mu), Float64(sigma), Float64(mu_sup), 10.0)
INormal_Uniform(mu, sigma) = INormal_Uniform(Float64(mu), Float64(sigma), Float64(mu))
Distributions.pdf(d::INormal_Uniform, x::Float64) = Distributions.pdf(Normal(d.mu, d.sigma), x)
Base.rand(d::INormal_Uniform) = rand(Uniform(d.mu_sup-d.sup/2, d.mu_sup+d.sup/2))
Base.rand(rng::AbstractRNG, d::INormal_Uniform) = rand(rng, Uniform(d.mu_sup-d.sup/2, d.mu_sup+d.sup/2))
Distributions.sampler(d::INormal_Uniform) = Distributions.sampler(Uniform(d.mu_sup-d.sup/2, d.mu_sup+d.sup/2))
Distributions.logpdf(d::INormal_Uniform, x::Real) = Distributions.logpdf(Normal(d.mu, d.sigma), x) 
Distributions.cdf(d::INormal_Uniform, x::Real) = Distributions.cdf(Normal(d.mu, d.sigma), x)
Distributions.quantile(d::INormal_Uniform, q::Real) = Distributions.quantile(Normal(d.mu, d.sigma), q)
Base.minimum(d::INormal_Uniform) = d.mu-d.sup/2
Base.maximum(d::INormal_Uniform) = d.mu+d.sup/2
Distributions.insupport(d::INormal_Uniform, x::Real) = Distributions.insupport(Normal(d.mu, d.sigma), x)
Distributions.mean(d::INormal_Uniform) = Distributions.mean(Normal(d.mu, d.sigma))
Distributions.var(d::INormal_Uniform) = Distributions.var(Normal(d.mu, d.sigma))

function Distributions.fit(::Type{<:INormal_Uniform}, x::AbstractArray{T}, w::AbstractArray{Float64}) where T<:Real
    norm_fit = Distributions.fit(Normal{T}, x, w)
    INormal_Uniform(norm_fit.μ, norm_fit.σ)
end

In [ ]:
struct INormal_GMM{T<:Real, C<:Categorical} <: ContinuousUnivariateDistribution
    mu::T
    sigma::T
    gmm_mu::Vector{T}
    gmm_sigma::Vector{T}
    gmm_prior::C
    INormal_GMM(mu::T, sigma::T, gmm_mu::AbstractVector{T}, gmm_sigma::AbstractVector{T}, gmm_prior::C) where {T<:Real,C<:Categorical} = new{T,C}(T(mu), T(sigma), gmm_mu, gmm_sigma, gmm_prior)
end

In [ ]:
INormal_GMM(mu, sigma) = INormal_GMM(Float64(mu), Float64(sigma), [Float64(mu), Float64(mu)], [sigma/2.0, sigma*2.0], Categorical(0.5, 0.5))
Distributions.pdf(d::INormal_GMM, x::Float64) = Distributions.pdf(Normal(d.mu, d.sigma), x)
Base.rand(d::INormal_GMM) = rand(UnivariateGMM(d.gmm_mu, d.gmm_sigma, d.gmm_prior))
Base.rand(rng::AbstractRNG, d::INormal_GMM) = rand(rng, UnivariateGMM(d.gmm_mu, d.gmm_sigma, d.gmm_prior))
Distributions.sampler(d::INormal_GMM) = Distributions.sampler(UnivariateGMM(d.gmm_mu, d.gmm_sigma, d.gmm_prior))
Distributions.logpdf(d::INormal_GMM, x::Real) = Distributions.logpdf(Normal(d.mu, d.sigma), x) 
Distributions.cdf(d::INormal_GMM, x::Real) = Distributions.cdf(Normal(d.mu, d.sigma), x)
Distributions.quantile(d::INormal_GMM, q::Real) = Distributions.quantile(Normal(d.mu, d.sigma), q)
Base.minimum(d::INormal_GMM) = -Inf
Base.maximum(d::INormal_GMM) = Inf
Distributions.insupport(d::INormal_GMM, x::Real) = Distributions.insupport(Normal(d.mu, d.sigma), x)
Distributions.mean(d::INormal_GMM) = Distributions.mean(Normal(d.mu, d.sigma))
Distributions.var(d::INormal_GMM) = Distributions.var(Normal(d.mu, d.sigma))

# function Distributions.fit(::Type{<:INormal_GMM}, x::AbstractArray{T}, w::AbstractArray{Float64}) where T<:Real
#     norm_fit = Distributions.fit(Normal{T}, x, w)
#     INormal_Uniform(norm_fit.μ, norm_fit.σ)
# end

#### Stress Testing Setup

In [ ]:
@with_kw struct AutoRiskParams
    endtime::Real = 30 # Simulate end time
end;

In [ ]:
@with_kw mutable struct AutoRiskSim <: GrayBox.Simulation
    t::Real = 0 # Current time
    params::AutoRiskParams = AutoRiskParams() # Parameters

    # System under test, ego vehicle
    sut = BlinkerVehicleAgent(get_ped_vehicle(id=1, s=5.0, v=15.0),
                              TIDM(ped_TIDM_template, noisy_observations=true))

    # Noisy adversary, pedestrian
    adversary = NoisyPedestrianAgent(get_pedestrian_noisy(id=2, s=7.0, v=2.0, noise=init_noise),
                                     AdversarialPedestrian())

    # Adversarial Markov decision process
    problem::MDP = AdversarialDrivingMDP(sut, [adversary], ped_roadway, 0.1)
    state::Scene = rand(initialstate(problem))
    prev_distance::Real = -Inf # Used when agent goes out of frame

    # Noise distributions and disturbances
#     xposition_noise::Distribution = INormal_Uniform(0, 1) # Gaussian noise (notice larger σ)
#     yposition_noise::Distribution = INormal_Uniform(0, 1) # Gaussian noise
    xposition_noise::Distribution = Normal(0, 1) # Gaussian noise (notice larger σ)
    yposition_noise::Distribution = Normal(0, 1) # Gaussian noise
    velocity_noise::Distribution = Normal(0, 1) # Gaussian noise
    
    # GPS range noise
    range_sigma = 3.0
    range_noise::Array{Distribution} = [Normal(0, range_sigma), Normal(0, range_sigma), Normal(0, range_sigma), Normal(0, range_sigma), Normal(0, range_sigma)] # Array of Gaussian
    
    disturbances = Disturbance[PedestrianControl()] # Initial 0-noise disturbance
end;

In [ ]:
function GrayBox.environment(sim::AutoRiskSim)
#     return GrayBox.Environment(:xpos => sim.xposition_noise,
#                                :ypos => sim.yposition_noise,
#                                :vel => sim.velocity_noise)

    
#     obs_history = sim.sut.model.observation_history
    
#     if length(obs_history[1])>0
#         base_meas = last(obs_history[1])
        
#         xpos_samp = Float64[]
#         ypos_samp = Float64[]
#         xwt_samp = Float64[]
#         ywt_samp = Float64[]
        
#         xprior = Normal(0.0, 5.0)
#         yprior = Normal(0.0, 5.0)
        
#         for i=1:100
#             temp_meas = Union{Missing, AdversarialDriving.GPSRangeMeasurement}[]
#             for j=1:length(sim.range_noise)
#                 cur_meas = base_meas[j]
#                 if typeof(cur_meas)==Missing
#                     continue
#                 else
#                     push!(temp_meas, AdversarialDriving.GPSRangeMeasurement(sat=cur_meas.sat, 
#                                                                             range=cur_meas.range, 
#                                                                             noise=rand(sim.range_noise[j])))
#                 end
#             end
#             gps_fix = GPS_fix(temp_meas)
#             true_pos = posg(sim.state[id(sim.adversary)])
#             prev_noise = noise(sim.state[id(sim.adversary)])
            

#             push!(xpos_samp, gps_fix[1]-true_pos.x)
#             push!(ypos_samp, gps_fix[2]-true_pos.y)
            
#             push!(xwt_samp, pdf(xprior, prev_noise.pos.x - (gps_fix[1]-true_pos.x)))
#             push!(ywt_samp, pdf(yprior, prev_noise.pos.y - (gps_fix[2]-true_pos.y)))
            
#         end
# #         @show xpos_samp, ypos_samp
#         sim.xposition_noise = Distributions.fit(INormal_Uniform{typeof(xpos_samp[1])}, xpos_samp, xwt_samp/sum(xwt_samp))
#         sim.yposition_noise = Distributions.fit(INormal_Uniform{typeof(ypos_samp[1])}, ypos_samp, ywt_samp/sum(ywt_samp))
# #         sim.xposition_noise = Distributions.fit(INormal_Uniform{typeof(xpos_samp[1])}, xpos_samp, ones(length(xpos_samp)))
# #         sim.yposition_noise = Distributions.fit(INormal_Uniform{typeof(ypos_samp[1])}, ypos_samp, ones(length(ypos_samp)))
# #         @show xposition_noise, yposition_noise
#     end
#     if 0.7 < sim.t
    prev_noise = noise(sim.state[id(sim.adversary)])
    
#     xposition_noise = INormal_Uniform(0, 5, prev_noise.pos.x)
#     yposition_noise = INormal_Uniform(0, 1, prev_noise.pos.y)
#     xposition_noise = INormal_GMM(0, 5)
#     yposition_noise = INormal_GMM(0, 1)
    xposition_noise = Normal(0, 5)
    yposition_noise = Normal(0, 1)
#     else
#         xposition_noise = sim.xposition_noise
#         yposition_noise = sim.yposition_noise
#     end
    return GrayBox.Environment(
                                 :vel => sim.velocity_noise,
#                                :range_1 => sim.range_noise[1],
#                                :range_2 => sim.range_noise[2],
#                                :range_3 => sim.range_noise[3],
#                                :range_4 => sim.range_noise[4],
#                                :range_5 => sim.range_noise[5]
                                 :xpos => xposition_noise,
                                 :ypos => yposition_noise
                                )
end

In [ ]:
function GrayBox.transition!(sim::AutoRiskSim, sample::GrayBox.EnvironmentSample)
    sim.t += sim.problem.dt # Keep track of time

#     println(sample)
    
    # replace current noise with new sampled noise
#     range_noise = [sample[:range_1].value, sample[:range_2].value, sample[:range_3].value, sample[:range_4].value, sample[:range_5].value]
#     range_noise = [sample[:range_1].value, 0.0, sample[:range_3].value, 0.0, 0.0]
#     noise = Noise(pos = (0.0, 0.0), vel = sample[:vel].value, gps_range = [0.0, 0.0, 0.0, 0.0, 0.0])
#     noise = Noise(pos = (0.0, 0.0), vel = 0.0, gps_range = range_noise)
    noise = Noise(pos = (sample[:xpos].value, sample[:ypos].value), vel = 0.0)
#     noise = Noise(pos = (sample[:xpos].value, sample[:ypos].value), vel = sample[:vel].value, gps_range = [0.0, 0.0, 0.0, 0.0, 0.0])
#     noise = Noise(pos = (sample[:xpos].value, sample[:ypos].value), vel = 0.0, gps_range = [0.0, 0.0, 0.0, 0.0, 0.0])
    sim.disturbances[1] = PedestrianControl(noise=noise)

    # step agents: given MDP, current state, and current action (i.e. disturbances)
    (sim.state, r) = @gen(:sp, :r)(sim.problem, sim.state, sim.disturbances)

    # return log-likelihood of actions, summation handled by `logpdf()`
    return logpdf(sample)::Real
end

In [ ]:
function BlackBox.initialize!(sim::AutoRiskSim)
    sim.t = 0
    sim.problem = AdversarialDrivingMDP(sim.sut, [sim.adversary], ped_roadway, 0.1)
    sim.state = rand(initialstate(sim.problem))
    sim.disturbances = Disturbance[PedestrianControl()] # noise-less
    sim.prev_distance = -Inf
end

In [ ]:
simx = AutoRiskSim()
BlackBox.initialize!(simx);

In [ ]:
out_of_frame(sim) = length(sim.state.entities) < 2 # either agent went out of frame

In [ ]:
function BlackBox.distance(sim::AutoRiskSim)
    if out_of_frame(sim)
        return sim.prev_distance
    else
        pedestrian, vehicle = sim.state.entities
        pos1 = posg(pedestrian)
        pos2 = posg(vehicle)
        return hypot(pos1.x - pos2.x, pos1.y - pos2.y)
    end
end

In [ ]:
begin
    envsample = rand(GrayBox.environment(simx))
    GrayBox.transition!(simx, envsample)
    BlackBox.distance(simx)
end

In [ ]:
function BlackBox.isevent(sim::AutoRiskSim)
    if out_of_frame(sim)
        return false
    else
        pedestrian, vehicle = sim.state.entities
        return collision_checker(pedestrian, vehicle)
    end
end

In [ ]:
function BlackBox.isterminal(sim::AutoRiskSim)
    return isterminal(sim.problem, sim.state) ||
           out_of_frame(sim) ||
           BlackBox.isevent(sim) ||
           sim.t ≥ sim.params.endtime
end

In [ ]:
function BlackBox.evaluate!(sim::AutoRiskSim, sample::GrayBox.EnvironmentSample)
    logprob::Real = GrayBox.transition!(sim, sample) # Step simulation
    d::Real       = BlackBox.distance(sim)           # Calculate miss distance
    event::Bool   = BlackBox.isevent(sim)            # Check event indication
    sim.prev_distance = d                            # Store previous distance
    return (logprob::Real, d::Real, event::Bool)
end

In [ ]:
begin
    envsample2 = rand(GrayBox.environment(simx))
    BlackBox.evaluate!(simx, envsample2) # (log-likelihood, distance, isevent)
end

##### Rollouts using CEM 

In [ ]:
function cem_rollout(mdp::ASTMDP, s::ASTState, d::Int64)
    cem_solver = CEMSolver(n_iterations=10,
                           num_samples=20,
                           episode_length=d,
                           show_progress=false)
    cem_mdp = mdp # deepcopy(mdp)
    prev_top_k = cem_mdp.params.top_k
    cem_mdp.params.top_k = 0
    cem_planner = solve(cem_solver, cem_mdp)
    q_value = 0
    is_distrs = convert(Vector{GrayBox.Environment}, search!(cem_planner, s), d)
    USE_MEAN = true # use the mean of the importance sampling distr, instead of rand.

    AST.go_to_state(mdp, s) # Records trace through this call

    for i in 1:length(is_distrs)
        is_distr = is_distrs[1]
        if USE_MEAN
            sample = mean(is_distr)
        else
            sample = rand(is_distr)
        end
        # @info sample
        # @info is_distr
        a::ASTAction = ASTSampleAction(sample)
        # a::ASTAction = ASTSampleAction(rand(GrayBox.environment(mdp.sim)))
        # AST.random_action(mdp)
        (s, r) = @gen(:sp, :r)(cem_mdp, s, a, Random.GLOBAL_RNG)
        q_value = r + discount(cem_mdp)*q_value
        # AST.go_to_state(mdp, s) # Records trace through this call
    end
    # AST.go_to_state(mdp, s) # Records trace through this call
    cem_mdp.params.top_k = prev_top_k

    return q_value
end

In [ ]:
function Base.convert(::Type{Vector{GrayBox.Environment}}, distr::Dict{Symbol, Vector{Sampleable}}, max_steps::Integer=1)
    env_vector = GrayBox.Environment[]
    for t in 1:max_steps
        env = GrayBox.Environment()
        for k in keys(distr)
            env[k] = distr[k][t]
        end
        push!(env_vector, env)
    end
    return env_vector::Vector{GrayBox.Environment}
end

In [ ]:
function Statistics.mean(d::Dict)
    meand = Dict()
    for k in keys(d)
        m = mean(d[k])
        meand[k] = GrayBox.Sample(m, logpdf(d[k], m))
    end
    return meand
end

In [ ]:
function AST.search!(planner::CEMPlanner, s::ASTState)
    mdp::ASTMDP = planner.mdp
    return action(planner, s)
end

###### AST setup

In [ ]:
function CrossEntropyMethod.cross_entropy_method(loss::Function,
                              d_in;
                              max_iter,
                              N=100,
                              elite_thresh = -0.99,
                              min_elite_samples = Int64(floor(0.1*N)),
                              max_elite_samples = typemax(Int64),
                              weight_fn = (d,x) -> 1.,
                              rng::AbstractRNG = Random.GLOBAL_RNG,
                              verbose = false,
                              show_progress = false,
                              add_entropy = (x)->x
                             )
    d = deepcopy(d_in)
    show_progress ? progress = CrossEntropyMethod.Progress(max_iter) : nothing

    for iteration in 1:max_iter
        # Get samples -> Nxm
        samples = rand(rng, d, N)
        @show samples 

        # sort the samples by loss and select elite number
        losses = [loss(d, s) for s in samples]
        order = sortperm(losses)
        losses = losses[order]
        N_elite = losses[end] < elite_thresh ? N : findfirst(losses .> elite_thresh) - 1
        N_elite = min(max(N_elite, min_elite_samples), max_elite_samples)

        verbose && println("iteration ", iteration, " of ", max_iter, " N_elite: ", N_elite)

        #update based on elite samples
        elite_samples = samples[order[1:N_elite]]
        weights = [weight_fn(d, s) for s in elite_samples]
        if all(weights .≈ 0.)
            println("Warning: all weights are zero")
        end
        d = fit(d, elite_samples, weights, add_entropy = add_entropy)
        show_progress && next!(progress)
    end
    d
end

In [ ]:
function setup_ast(seed=0)
    # Create gray-box simulation object
    sim::GrayBox.Simulation = AutoRiskSim()

    # AST MDP formulation object
    mdp::ASTMDP = ASTMDP{ASTSampleAction}(sim)
    mdp.params.debug = true # record metrics
    mdp.params.top_k = 10   # record top k best trajectories
    mdp.params.seed = seed  # set RNG seed for determinism
    
#     function null_priority(mdp, s, snode) # snode is the state node of type DPWStateNode
# #         @show snode.tree.a_lookup
# #         Replace with null action calculated from noise-free measurements
#         null_action = ASTSampleAction(
#                                     GrayBox.EnvironmentSample(
#                                                 :vel => GrayBox.Sample(0., logpdf(Normal(0., 1), 0.)),
#                                                 :xpos => GrayBox.Sample(0., logpdf(Normal(0., 5.), 0.)),
#                                                 :ypos => GrayBox.Sample(0., logpdf(Normal(0., 1.), 0.))
#                                                 )
#                                     )
#         n_children = length(snode.tree.children[snode.index])
#         if n_children > 0
#             new_action = rand(actions(mdp, s))  # add a random action
#         else
#             new_action = null_action
#         end
# #         @show new_action.sample
#         return new_action
#     end
#     function prior_Q(mdp, s, a)
#         l1_norm = 0
#         for (key, val) in a.sample
#             l1_norm = l1_norm + abs(val.value)
#         end
#         return 100.
#     end


#     # Hyperparameters for MCTS-PW as the solver
#     solver = MCTSPWSolver(n_iterations=1000,        # number of algorithm iterations
#                           exploration_constant=1.0, # UCT exploration
#                           k_action=1.0,             # action widening
#                           alpha_action=0.5,         # action widening
#                           depth=sim.params.endtime, # tree depth
# #                           init_Q=100.,
# #                           init_N=1,
#                           next_action=null_priority
# #                           estimate_value=cem_rollout
#                          )
    solver = CEMSolver(n_iterations=2,
                       num_samples=1,
                       episode_length=sim.params.endtime)
    
#     return actions(mdp, initialstate(mdp))
    
    # Get online planner (no work done, yet)
    planner = solve(solver, mdp)

    return planner
end;

In [ ]:
planner = setup_ast();

In [ ]:
action_trace = search!(planner)

In [ ]:
planner.is_dist[:xpos]

In [ ]:
episodic_figures(planner.mdp, gui=false); POMDPStressTesting.gcf();

In [ ]:
distribution_figures(planner.mdp, gui=false); POMDPStressTesting.gcf();

In [ ]:
playback_trace = playback(planner, action_trace, BlackBox.distance, return_trace=true)

In [ ]:
failure_rate = print_metrics(planner)

In [ ]:
begin
    # TODO: get this index from the `trace` itself
    # findmax(planner.mdp.metrics.reward[planner.mdp.metrics.event])
    # findmax(ast_mdp.metrics.reward[ast_mdp.metrics.event])

    failure_likelihood =
        round(exp(maximum(planner.mdp.metrics.logprob[planner.mdp.metrics.event])), digits=8)

    Markdown.parse(string("\$\$p = ", failure_likelihood, "\$\$"))
end

In [ ]:
playback_trace = playback(planner, action_trace, sim->sim.state, return_trace=true)

In [ ]:
# t = 10
mp = @manipulate for t=slider(1:30, value=1., label="t")
    AutomotiveVisualization.render([planner.mdp.sim.problem.roadway, crosswalk, playback_trace[min(t, length(playback_trace))]])
end

In [ ]:
planner.tree.q

### Example for localizing using GPS

**Input**: state of agent, noise in each GPS range measurement

**Output**: agent with noise parameter containing difference between estimated state and true state

In [ ]:
eg1_meas = AdversarialDriving.measure_gps(ped_state, [0.0, 0.0, 0.0, 0.0])

In [ ]:
ped_state.state.veh_state

In [ ]:
eg1_ped = AdversarialDriving.localize(ped_state, [eg1_meas], planner.mdp.sim.problem.roadway)

### Example for Distribution likelihood estimation under measurements (In progress)

**Input**: distribution of measurements, function mapping from measurements to states of all agents

**Output**: log probability of a given set of agent states 

In [ ]:
eg2_range_noise = [Normal(0, 5.0), Normal(0, 5.0), Normal(0, 5.0)] # Array of Gaussian
eg2_range_mu = [10.0, 11.0, 9.0]

In [ ]:
function eg2_localize(m)
    return [1/sqrt(2)*hypot(m[1], m[2]), 1/sqrt(2)*hypot(m[2], m[3])]
end

In [ ]:
eg2_localize([1.1, 1.3, 2.8])

In [ ]:
n_samp = 1000
eg2_noise_samp = zeros(n_samp, length(eg2_range_noise))
for i=1:length(eg2_range_noise)
    eg2_noise_samp[:, i] = rand(eg2_range_noise[i], n_samp)
end
eg2_noise_samp

In [ ]:
loc_samp = zeros(n_samp, 2)
for i=1:n_samp
    loc_samp[i, :] = eg2_localize(eg2_range_mu + eg2_noise_samp[i, :])
end
loc_samp

In [ ]:
using Plots

In [ ]:
histogram(loc_samp[:, 1])
histogram!(loc_samp[:, 2])

Need to calc log likelihood from the empirical distribution